In [ ]:
using Pkg
Pkg.add("DelimitedFiles")
Pkg.add("DSP")

using DelimitedFiles
csvFile = ".\\data\\data.csv"

# 读取csv文件
csvData = DelimitedFiles.readdlm(csvFile, ',', header=true)
data = csvData[1]

# 读取加速度计的数据
accX = data[:, 22]
accY = data[:, 23]
accZ = data[:, 24]

#时间轴
timeInterval = 0.03 # 每个数据点之间的时间间隔（假设为1秒）
numData = size(data, 1) #数据点的总数
time = (0:numData-1) * timeInterval

# 将加速度计的数据进行可视化
figure(1)
plot(time, accX)
hold("on")
plot(time, accY)
hold("on")
plot(time, accZ)
hold("off")
title("加速度计的数据")
xlabel("时间（秒）")
ylabel("加速度（m/s^2）")
legend(["X方向", "Y方向", "Z方向"])

figure(2)
plot(time[1:500], accX[1:500])
hold("on")
plot(time[1:500], accY[1:500])
hold("on")
plot(time[1:500], accZ[1:500])
hold("off")
xlabel("时间（秒）")
ylabel("加速度（m/s^2）")
legend(["X方向", "Y方向", "Z方向"])
title("前500个数据的加速度数据变化")

#计算整体加速度
figure(3)
mag = sqrt.(sum(accX .^ 2 + accY .^ 2 + accZ .^ 2, dims=2))
plot(time[1:500], mag[1:500])
xlabel("时间（秒）")
ylabel("整体加速度（m/s^2）")
title("前500个数据的整体加速度数据变化")

# 计算去除重力影响后的加速度
magNoG = mag .- mean(mag[2:end])

# 计算峰值阈值
minPeak = std(magNoG[2:end])

# 使用 findpeaks 函数寻找峰值
# 使用 DSP.jl 包的 findpeaks 函数寻找峰值
pks, locs = findpeaks(magNoG)

# 根据峰值阈值过滤峰值
filteredLocs = locs[findall(x -> x >= minPeak, pks)]
filteredpks = pks[findall(x -> x >= minPeak, pks)]

# 计算步数
numSteps = length(filteredpks)

# 显示步数
println("估计的步数为：", numSteps)

# 可视化峰值位置和幅值
figure(4)
plot(time, magNoG, label="加速度幅值，去除重力影响（m/s^2）")
hold("on")
scatter(time[filteredLocs], filteredpks, label="峰值", color=:red, marker=:v, linestyle=:none)
hold("off")
title("步数估计")
xlabel("时间（秒）")
ylabel("加速度幅值，去除重力影响（m/s^2）")

# 只选择前五百个点
time500 = time[1:500]
magNoG500 = magNoG[1:500]
filteredLocs500 = filteredLocs[filteredLocs.<=500]
filteredpks500 = filteredpks[filteredLocs.<=500]

# 可视化峰值位置和幅值
figure(5)
plot(time500, magNoG500, label="加速度幅值，去除重力影响（m/s^2）")
hold("on")
scatter(time500[filteredLocs500], filteredpks500, label="峰值", color=:red, marker=:v, linestyle=:none)
hold("off")
title("步数估计")
xlabel("时间（秒）")
ylabel("加速度幅值，去除重力影响（m/s^2）")